In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from glob import glob
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import math
from sklearn.preprocessing import LabelEncoder

RANDOM_SEED = 42
NUM_CLASSES = 5


"""
buy_post - целевая переменная
date_diff_post - целевая переменная
"""

'\nbuy_post - целевая переменная\ndate_diff_post - целевая переменная\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
url = os.chdir(r"/content/drive/MyDrive/hackatonZP/")

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/hackatonZP/main_train.csv')

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/hackatonZP/main_test.csv')
df_test

,Unnamed: 0,index,customer_id,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval
0,0,0,13220760,666,40.141587,175.790476,8,607,178,174.40,74.10,2.00,419.98,-15.761007,0,11,-0.991379
1,1,1,30315975,789,142.490000,235.000000,1,789,845,235.00,235.00,584.96,144.98,-26.936829,0,3,0.000000
2,2,2,21679985,666,48.471053,209.526316,7,666,666,165.00,165.00,274.98,349.99,16.060093,0,8,5.267212
3,3,3,29754274,899,112.202632,98.421053,7,899,833,150.00,80.00,509.96,690.95,-21.969896,0,4,-5.492474
4,4,4,7797823,666,71.571529,114.500000,7,94,666,114.50,114.50,468.97,520.96,11.252500,0,12,2.748228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112329,112329,112329,14023374,469,104.990000,105.200000,7,469,803,105.20,105.20,99.99,44.99,-19.909722,0,3,-15.381258
112330,112330,112330,34923292,925,174.976667,241.733333,6,925,925,227.50,270.20,149.98,224.97,25.796794,0,3,-0.978383
112331,112331,112331,32761835,898,146.747500,123.640000,7,898,458,115.46,109.50,99.00,199.00,4.847801,0,4,1.239201
112332,112332,112332,38322785,478,233.656667,117.550000,8,478,956,117.55,117.55,599.98,99.99,0.013113,0,3,0.001331


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import  RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
X, y = df_train.drop(columns=['customer_id', 'buy_post', 'index']),df_train['buy_post']

In [ ]:
X = X.drop(columns=['Unnamed: 0','popular_dish_product'])
X = X[['popular_dish','avg_revenue','avg_sqm','popular_format','m1_dish','m2_dish','m1_sqm','m2_sqm','m1_revenue','m2_revenue','m2_interval','mean_delta_time','visits','mean_interval']]
X

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval
0,47.0,149.627941,439.029412,6,589,625,300.00,300.00,389.97,119.99,0.100428,0,25,2.316445
1,588.0,93.042623,320.000000,2,632,461,320.00,320.00,99.99,399.99,0.192454,0,25,2.079109
2,900.0,109.227791,153.000000,7,786,857,153.00,153.00,49.99,99.98,0.122477,0,24,2.091698
3,857.0,51.490000,128.884000,3,510,858,139.00,139.00,344.97,49.97,0.012164,0,4,6.960298
4,87.0,66.593200,171.474400,7,899,900,129.27,129.27,604.93,554.94,7.181852,0,3,5.149244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,978.0,85.081818,112.530000,7,978,720,112.53,112.53,514.95,99.99,0.014687,0,3,0.407315
499996,900.0,92.297692,120.000000,8,987,900,120.00,120.00,349.96,49.99,0.008623,0,3,0.005027
499997,847.0,99.083636,80.200000,7,847,847,80.20,80.20,169.99,399.97,0.048738,0,3,0.047388
499998,656.0,203.321667,270.000000,6,656,652,270.00,270.00,469.96,179.99,0.076979,0,3,0.083773


In [ ]:
X_test = df_test.drop(columns=['customer_id','index','Unnamed: 0'])
X_test

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval
0,666,40.141587,175.790476,8,607,178,174.40,74.10,2.00,419.98,-15.761007,0,11,-0.991379
1,789,142.490000,235.000000,1,789,845,235.00,235.00,584.96,144.98,-26.936829,0,3,0.000000
2,666,48.471053,209.526316,7,666,666,165.00,165.00,274.98,349.99,16.060093,0,8,5.267212
3,899,112.202632,98.421053,7,899,833,150.00,80.00,509.96,690.95,-21.969896,0,4,-5.492474
4,666,71.571529,114.500000,7,94,666,114.50,114.50,468.97,520.96,11.252500,0,12,2.748228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112329,469,104.990000,105.200000,7,469,803,105.20,105.20,99.99,44.99,-19.909722,0,3,-15.381258
112330,925,174.976667,241.733333,6,925,925,227.50,270.20,149.98,224.97,25.796794,0,3,-0.978383
112331,898,146.747500,123.640000,7,898,458,115.46,109.50,99.00,199.00,4.847801,0,4,1.239201
112332,478,233.656667,117.550000,8,478,956,117.55,117.55,599.98,99.99,0.013113,0,3,0.001331


In [ ]:
y

0         1
1         1
2         1
3         1
4         1
         ..
499995    0
499996    0
499997    1
499998    1
499999    0
Name: buy_post, Length: 500000, dtype: int64

In [ ]:
X = X.fillna(-1)
y = y.fillna(-1)

In [ ]:
y = np.array(y)
y

array([1, 1, 1, ..., 1, 1, 0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=42)

In [ ]:
y_reg_train = X_train['date_diff_post']
X_train = X_train.drop('date_diff_post',axis = 1)
y_reg_test = X_test['date_diff_post']
X_test = X_test.drop('date_diff_post',axis = 1)

In [ ]:
model = HistGradientBoostingClassifier()

model.fit(X,y)

HistGradientBoostingClassifier()

In [26]:
X_reg, y_reg = df_train.drop(columns=['customer_id', 'index']),df_train['buy_post']

In [27]:
X_reg = X_reg[X_reg['buy_post']==1]
X_reg = X_reg.drop(columns=['Unnamed: 0','popular_dish_product','buy_post'])
X_reg = X_reg[['popular_dish','avg_revenue','avg_sqm','popular_format','m1_dish','m2_dish','m1_sqm','m2_sqm','m1_revenue','m2_revenue','m2_interval','mean_delta_time','visits','mean_interval']]
X_reg

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval
0,47.0,149.627941,439.029412,6,589,625,300.00,300.00,389.97,119.99,0.100428,0,25,2.316445
1,588.0,93.042623,320.000000,2,632,461,320.00,320.00,99.99,399.99,0.192454,0,25,2.079109
2,900.0,109.227791,153.000000,7,786,857,153.00,153.00,49.99,99.98,0.122477,0,24,2.091698
3,857.0,51.490000,128.884000,3,510,858,139.00,139.00,344.97,49.97,0.012164,0,4,6.960298
4,87.0,66.593200,171.474400,7,899,900,129.27,129.27,604.93,554.94,7.181852,0,3,5.149244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499990,987.0,66.187500,250.000000,2,588,978,250.00,250.00,501.00,259.00,0.773773,0,3,0.308032
499993,588.0,121.913077,257.153846,2,620,588,250.00,268.60,544.96,644.95,1.050243,0,3,0.660340
499994,1020.0,187.490000,244.500000,6,899,899,244.50,244.50,864.96,569.96,0.008657,0,4,0.214685
499997,847.0,99.083636,80.200000,7,847,847,80.20,80.20,169.99,399.97,0.048738,0,3,0.047388


In [31]:
y_reg = df_train[['date_diff_post', 'buy_post']]
y_reg = y_reg.fillna(-1)
y_reg = y_reg[y_reg['buy_post']==1]
y_reg = y_reg.drop('buy_post',axis=1)
y_reg

,date_diff_post
0,9.0
1,10.0
2,4.0
3,42.0
4,9.0
...,...
499990,3.0
499993,1.0
499994,55.0
499997,10.0


In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model = RandomForestRegressor()

model.fit(X_reg,y_reg)

mean_squared_error(model.predict(X_reg),y_reg)

<ipython-input-32-e87ddeff40fd>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_reg,y_reg)


27.866964548657037

In [35]:
model.predict(X_reg[:100])

array([ 7.78, 10.66,  5.78, 38.33, 18.74, 32.72, 10.62,  5.82, 13.57,
       30.77, 21.92, 21.57, 34.4 , 21.28, 23.09, 14.5 , 14.54, 14.48,
        8.63, 19.95, 11.39, 34.83,  8.93, 23.8 , 41.51, 37.1 ,  9.24,
        5.9 , 18.77,  6.87, 16.  , 37.58, 20.93, 10.05, 37.02, 35.  ,
       23.13, 23.06, 18.2 , 21.72, 34.35, 17.55, 31.63,  5.37, 39.22,
        8.57, 14.84,  3.3 , 29.37, 23.57, 14.04, 13.5 , 14.06, 38.8 ,
       33.49, 25.77, 24.01, 13.84, 23.46, 34.53,  5.54, 19.51,  2.9 ,
       34.34, 25.78, 13.26, 13.82, 11.09,  9.88, 30.75, 15.45,  2.39,
        9.15, 35.6 , 26.72, 36.73, 39.44, 24.96, 24.98, 27.15,  9.81,
       25.16, 17.92, 44.55, 11.21, 13.48, 48.29, 14.44, 15.76,  6.93,
        7.89, 41.32, 13.76, 31.27, 20.3 , 23.75, 19.94, 28.81, 33.37,
       17.5 ])

In [33]:
from joblib import dump, load
dump(model, 'regressor.joblib')

['regressor.joblib']

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(model.predict(X),y))

0.8381592490478577


In [ ]:
from joblib import dump, load
dump(model, 'filename.joblib')

['filename.joblib']

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
df_ans = pd.DataFrame(y_pred)
df_ans

,0
0,0
1,0
2,1
3,0
4,1
...,...
112329,0
112330,0
112331,0
112332,0


In [ ]:
df_ans.to_csv('ihateyou.csv')

Начинаем регрессию

In [ ]:
X['buy_post'] = pred
X['date_diff_post'] = df_train['date_diff_post']

In [ ]:
X = X[X['buy_post']==1]
X

In [ ]:
X['date_diff_post'].unique()

In [ ]:
import numpy as np
import math

X['date_diff_post'] = X['date_diff_post'].fillna(X['date_diff_post'].mean())


In [ ]:
X

In [ ]:
X = X.drop(columns=['buy_post'])

In [ ]:
X

In [ ]:
X.to_csv(r'train_2.csv', index = False)

train_2 - файл готовый для обучения регрессии

In [ ]:
train = pd.read_csv('train_2.csv')

In [ ]:
train.columns

In [ ]:
train['date_diff_post'].mean()

In [ ]:
df = train[train['date_diff_post']==train['date_diff_post'].mean()]

In [ ]:
help = train[train['date_diff_post']!=train['date_diff_post'].mean()]

Работаем с date_diff_post

In [ ]:
X, y = help.drop(columns=['date_diff_post','popular_dish_product']), help['date_diff_post']

In [ ]:
X

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

model = RandomForestRegressor()

model.fit(X,y)

mean_squared_error(model.predict(X),y)